<a href="https://colab.research.google.com/github/AnSer04/MyHashTable/blob/main/L_Translator_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовка текстовых данных для обучения



**Парсинг исходного файла словаря обучающей выборки и формирование sources и targets**

1. Открытие архива словаря WordTheme на диске
2. Чтение файла json из архива
3. Разбор файла и выделение обучающих примеров
4. Отделение исходных и целевых текстов в отдельные списки для дальнейшей работы с ними

In [ ]:
import zipfile as ziplib
import json
import pandas as pd

# Пути к файлам
word_theme_zip_file_path = "/content/drive/MyDrive/WordTheme/Корпус переводов на лиасфирский_20240430_1431.wt"
dictionary_file = "dictionary.txt"

# Чтение содержимого из zip файла
with ziplib.ZipFile(word_theme_zip_file_path, "r") as zip_file:
  with zip_file.open(dictionary_file) as file:
    file_content = file.read().decode("utf-8")

# Выделение обучающих примеров в один список
train_samples = [(item["m"], item["t"]) for item in json.loads(file_content)["lword"]]

# Формирование исходных и целевых списков для обучения
sources = [sample[0] for sample in train_samples]
targets = [sample[1] for sample in train_samples]

# Визуализация данных для теста
pd.DataFrame(
    train_samples,
    columns=["Русский", "Лиасфирский"]
)

,Русский,Лиасфирский
0,"Здравствуйте! Очень рад, что смог вас застать ...","Al ū'sarezi! Af taÿ ki dorivaÿ az, st'uaksoėo ..."
1,"Здравствуйте! Очень рада, что смогла вас заста...","Al ū'sarezi! Af taï ki dorivaï, st'uaksao oaėt..."
2,"Всем привет! Я тут новый, поэтому хочу со всем...","Ū'sarezi vöė-ū's! Af taÿ ki fenÿ ügup, aqaėo ĺ..."
3,"Привет, ребята! Как вас зовут? Я тут совсем но...","Ū'sarezi, paksië! Af önze uolaf akli? Af taï k..."
4,"Здравствуйте, как ваше имя?","Ū'sarezi, af al öni uola akli?"
...,...,...
2529,"Они мне надоели, поэтому я решил уйти, а они п...","Hateėo voz vöė-voz, aupazeh veĺ, ūatea taÿ oėo..."
2530,"Они прожили вместе долгую и прекрасную жизнь, ...","Inuaa voz le-nua ki beoai a soni ū'i, afizai v..."
2531,"Я чувствоваю такое же, что и ты чувствовала то...","Aėşeėo taÿ ho oks, a aėşea taoï ola ve-oaĺ. — ..."
2532,Жил-был мальчик по имени Амир. Он жил в малень...,Afiza rolavo apeėli lö li-uola Amir. Nuaa voa ...


**Настройка и обучение токенизаторов source_tokenizer и target_tokenizer**

1. Инициализация токенизаторов
2. Настройка для каждого токенизатора:
    - предтокенизатора
    - нормализатора токенов
    - параметров постобработки последовательностей
    - параметров обучение токенизаторов
    - декодера для преобразования в текст на целевом языке
3. Обучение токенизаторов
4. Паддинг данных

In [ ]:

from tokenizers import Tokenizer, Regex
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer
from tokenizers import normalizers
from tokenizers.pre_tokenizers import Split
from tokenizers.normalizers import Lowercase, Replace
from tokenizers.processors import TemplateProcessing
from tokenizers import decoders


# Инициализация токенизаторов
source_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]", dropout=0.1))
target_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]", dropout=0.1))


# Настройка исходного токенизатора
split_pattern = Regex(r"\d|[\w']+|[.,!?;:—\-]")

source_tokenizer.pre_tokenizer = Split(pattern=split_pattern, behavior="removed", invert=True)
source_tokenizer.normalizer = normalizers.Sequence([
    Lowercase(),
    Replace("-", " "),
    Replace("—", "-"),
    Replace("ё", "е")
])

source_tokenizer.post_processor = TemplateProcessing(
    single="[SOS] $A [EOS]",
    special_tokens=[
        ("[SOS]", 2),
        ("[EOS]", 3),
    ],
)

source_trainer = WordPieceTrainer(special_tokens=["[PAD]", "[UNK]", "[SOS]", "[EOS]"], vocab_size=2000, min_frequency=2)
source_tokenizer.decoder = decoders.WordPiece()

# Настройка целевого токенизатора
split_pattern = Regex(r"\d|[\w'\-]+|[.,!?;:—]")

target_tokenizer.pre_tokenizer = Split(pattern=split_pattern, behavior="removed", invert=True)
target_tokenizer.normalizer = normalizers.Sequence([
    Lowercase(),
    Replace("—", "-")
])

target_tokenizer.post_processor = TemplateProcessing(
    single="[SOS] $A [EOS]",
    special_tokens=[
        ("[SOS]", 2),
        ("[EOS]", 3),
    ],
)

target_trainer = WordPieceTrainer(special_tokens=["[PAD]", "[UNK]", "[SOS]", "[EOS]"], vocab_size=2000, min_frequency=2)
target_tokenizer.decoder = decoders.WordPiece()

# Обучение токенизаторов с нормализованными данными
source_tokenizer.train_from_iterator(sources, trainer=source_trainer)
target_tokenizer.train_from_iterator(targets, trainer=target_trainer)

# Определение максимальной длины последовательности для паддинга
source_max_length = max(len(source_tokenizer.encode(text).ids) for text in sources)
target_max_length = max(len(source_tokenizer.encode(text).ids) for text in targets)
max_length = max(source_max_length, target_max_length)

# Паддинг
source_tokenizer.enable_padding(max_length=max_length, pad_id=0, pad_token="[PAD]")
target_tokenizer.enable_padding(max_length=max_length, pad_id=0, pad_token="[PAD]")

**Небольшой тест токенизаторов**

In [ ]:

print(f"Source Tokenizer: {source_tokenizer.get_vocab_size()} tokens")
print(f"Target Tokenizer: {target_tokenizer.get_vocab_size()} tokens")

# Тест исходного токенизаторsа
source_output = source_tokenizer.encode_batch([
    "Привет, я \"твой\" переводчик, который поможет тебе познакомиться с лиасфирским.",
    "Я очень стараюсь научиться очень хорошо переводить на лиасфирский — прости если что.",
    "Мечта, творчество - вот ключ к успеху, силе и чему-то новому!",
    "Ёжик плыл по реке тихо-тихо и думал, где он сейчас.",
    "У меня есть 123 яблока и 456 груш.",
    "Почему так тяжело, просто переводить или перевести что-то на лиасфирский?",
    "Я люблю тебя, а ты любишь ли меня?",
    "Смог бы кто-нибудь помочь мне выбраться оттуда?",
    "Собрать, выбрать, забрать, убрать, крыть, открыть, закрыть, укрыть, раскрыть, покрыть, покрытие"

])

def pretty_tokenized_print(encoded_text, tokenizer):
    print(f"\n[{len(encoded_text.tokens)} tokens in length]")
    print(f"INPUT: {tokenizer.decode(encoded_text.ids)}")
    tokens = encoded_text.tokens
    ids = [id for id in encoded_text.ids if id > 0]
    if "[PAD]" in tokens:
        index = tokens.index("[PAD]")
        tokens = tokens[:index] + ["..."]
    print("TOKENS:", *tokens)
    print("IDs:", ids)



for encoded_text in source_output:
    pretty_tokenized_print(encoded_text, source_tokenizer)

print("-"*40)

# Тест целевого токенизатора
target_output = target_tokenizer.encode_batch([
    "Ū'sarezi, aėni, af taÿ öni lauoĺin ki anaėsehoė vöė-taoÿ apūaėvah le-liasfir",
    "Neėvaėo taÿ az, st'uaksavo lauoĺh laė-liasfir leĺ, se naėni ö vo-oaĺvo ki lėo",
    "Faĺ, vellofas: af o tos ki avifosai ksoĺ li-anaėa, li-raĺ a ksoĺ vi-oaĺvo ki feni evaėvos!",
    "Faävai voa assioli a ūavai, af voa üva st'aė",
    "Ünaėo taÿ 123 le-habruz a 456 le-hrosïz.",
    "Af e ki eraĺi ho uahaėo, lauoĺh laė-liasfir?",
    "Ūaė taÿ ve-taoï, ae taoï ūaėö taÿk",
    "St'uaksavo anaėseh goaĺvo vöė-taoï, evah eźźiú?",
    "Af taÿ tefak naėcoh onoė ve-oaĺvo vu. ifo avifosaÿ",
    "Aĺzeh, öuzeh, oaėzeh, faėzeh, ezeh, uazeh, uazen, laorozeh, tueėzeh, ueklizeh, hekzeh, aėzeh, uėzeh",
    "Lele-ūab, lele-oaė, le-öu, le-oaė, tavo, tavoÿ, tavoï, tavoi, tavoin, tavoinz, tavoinaë, ū'sarezi, ū'sareziaë, ū'sareziz, ū'sarezih, ū'sareziao, ū'sarezihoė, ū'sarezihoz"
])

for encoded_text in target_output:
   pretty_tokenized_print(encoded_text, target_tokenizer)

Source Tokenizer: 2000 tokens
Target Tokenizer: 2000 tokens

[94 tokens in length]
INPUT: привет, я твой переводчик, который поможет тебе познакомиться с лиасфирским.
TOKENS: [SOS] привет , я твой перево ##д ##чи ##к , который поможет тебе познакомиться с лиасфирс ##ким . [EOS] ...
IDs: [2, 369, 5, 50, 746, 1581, 67, 190, 72, 5, 327, 1200, 246, 755, 36, 1062, 1414, 7, 3]

[94 tokens in length]
INPUT: я очень стараюсь научиться очень хорошо переводить на лиасфирский - прости если что.
TOKENS: [SOS] я очень ста ##ра ##юсь на ##учи ##ться очень хорошо перево ##дить на лиасфирс ##кий - прости если что . [EOS] ...
IDs: [2, 50, 208, 469, 101, 596, 92, 1171, 242, 208, 399, 1581, 603, 92, 1062, 493, 6, 673, 285, 97, 7, 3]

[94 tokens in length]
INPUT: мечта, творчество вот ключ к успеху, силе и чему то новому!
TOKENS: [SOS] мечт ##а , тво ##р ##че ##ство вот к ##лю ##ч к у ##с ##пе ##х ##у , сил ##е и чем ##у то ново ##му ! [EOS] ...
IDs: [2, 1459, 58, 5, 238, 57, 340, 348, 521, 29, 241, 66, 2

**Создание датасетов и загрузчиков данных train_data_loader и val_data_loader для модели**

1. Создание класса датасета, наследуясь от датасета из торча
2. Создание функции разделение на обучающую и валидационные выборки
3. Создание датасета для входных данных
4. Формирование загрузчиков данных

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split

class MyDataset(Dataset):
    def __init__(
        self,
        sources: list[str],
        targets: list[str],
        source_tokenizer: Tokenizer,
        target_tokenizer: Tokenizer
    ):
        # Преобразование исходных и целевых последовательностей в индексы токенов
        self.sources = [sequence.ids for sequence in source_tokenizer.encode_batch(sources)]
        self.targets = [sequence.ids for sequence in target_tokenizer.encode_batch(targets)]

    def __len__(self):
        return len(self.sources)

    def __getitem__(self, index: int) -> dict:
        # Получение одного образца данных из набора данных по индексу
        source = self.sources[index]
        target = self.targets[index]
        return {
            "input_sequence": source,
            "target_sequence": target
        }

def split_train_test(dataset: MyDataset, batch_size=32, validation_split=0.2) -> tuple[DataLoader, DataLoader]:
    # Разделение набора данных на обучающую и валидационную выборки
    total_samples = len(dataset)
    val_size = int(validation_split * total_samples)
    train_size = total_samples - val_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    # Создание загрузчиков данных для обучения и валидации
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
    return train_data_loader, val_data_loader

# Создание экземпляра набора данных
dataset = MyDataset(sources, targets, source_tokenizer, target_tokenizer)

# Разделение данных на обучающую и валидационную выборки с помощью функции split_train_test
train_data_loader, val_data_loader = split_train_test(dataset)

# Создание модели

**Подготовка данных: важные переменные**

- `sources` - список исходных текстов на русском
- `targets` - список переводов текстов на лиасфирский
- `source_tokenizer` - токенизатор для исходного языка (русский)
- `target_tokenizer` - токенизатор для целевого языка перевода (лиасфирский)
- `dataset` - набор данных, исходные и целевые, преобразованные в числовые идентификаторы
- `train_data_loader` - загрузчики данных для основного обучения, тексты перемешаны и распределены по батчам (с моделью)
- `val_data_loader` - для валидации (с моделью)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, dropout=dropout)

    def forward(self, x):
        embedding = self.embedding(x)  # Преобразование входных токенов в векторные представления
        outputs, hidden = self.rnn(embedding)  # Передача векторов через RNN
        return outputs, hidden  # Возвращение выходов и последнего скрытого состояния

In [ ]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))

    def forward(self, hidden, encoder_outputs):
        timestep = encoder_outputs.shape[0]
        hiddens = hidden.repeat(timestep, 1, 1).transpose(0, 1)
        encoder_outputs = encoder_outputs.transpose(0, 1)
        energy = torch.tanh(self.attn(torch.cat((hiddens, encoder_outputs), dim=2)))
        attention = torch.softmax(torch.sum(self.v * energy, dim=2), dim=1).unsqueeze(1)
        return attention

In [ ]:
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout, attention):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.GRU(hidden_size + embedding_size, hidden_size, num_layers, dropout=dropout)
        self.fc = nn.Linear(hidden_size * 2, output_size)
        self.attention = attention

    def forward(self, x, hidden, encoder_outputs):
        x = x.unsqueeze(0)
        embedding = self.embedding(x)  # Преобразование целевых токенов в векторные представления
        attention = self.attention(hidden, encoder_outputs)  # Вычисление весов внимания
        attention_weights = torch.bmm(attention, encoder_outputs.transpose(0, 1))
        attention_weights = attention_weights.transpose(0, 1)
        rnn_input = torch.cat((embedding, attention_weights), dim=2)  # Объединение входных токенов с контекстным вектором
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        output = output.squeeze(0)
        attention_weights = attention_weights

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(target_tokenizer.word2index)
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
        encoder_outputs, hidden = self.encoder(source)
        x = target[0]
        for t in range(1, target_len):
            output, hidden = self.decoder(x, hidden, encoder_outputs)
            outputs[t] = output
            best_guess = output.argmax(1)
            x = target[t] if random.random() < teacher_force_ratio else best_guess
        return outputs

In [ ]:
# Параметры модели

input_size = source_tokenizer.get_vocab_size()
output_size = target_tokenizer.get_vocab_size()
embedding_size = 256
hidden_size = 512
num_layers = 2
dropout = 0.5
learning_rate = 0.001
pad_index = target_tokenizer.encode("[PAD]", add_special_tokens=False)  # Индекс для паддинга
start_index = target_tokenizer.encode("[SOS]", add_special_tokens=False)  # Индекс начала последовательности
end_index = target_tokenizer.encode("[EOS]", add_special_tokens=False)  # Индекс конца последовательности

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(input_size, embedding_size, hidden_size, num_layers, dropout).to(device)
attention = Attention(hidden_size).to(device)
decoder = Decoder(output_size, embedding_size, hidden_size, output_size, num_layers, dropout, attention).to(device)

model = Seq2Seq(encoder, decoder).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

print(model)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(2000, 256)
    (rnn): GRU(256, 512, num_layers=2, dropout=0.5)
  )
  (decoder): Decoder(
    (embedding): Embedding(2000, 256)
    (rnn): GRU(768, 512, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=2000, bias=True)
    (attention): Attention(
      (attn): Linear(in_features=1024, out_features=512, bias=True)
    )
  )
)
